# Install and load all libraries

In [3]:
from ipywidgets import Video, Image
from IPython.display import display
from IPython.display import HTML
from fastai.vision.all import *
from fastcore.parallel import *
from tqdm.auto import tqdm
import zipfile
import shutil
import timm
import cv2
import os

# Download and unzip the data

Make sure your current directory is `surgtoolloc`

In [ ]:
path=Path("./data")
path.mkdir(exist_ok=True)
# os.chdir(path)

In [ ]:
# !wget https://console.cloud.google.com/storage/browser/isi-surgtoolloc-2022?pli=1

In [2]:
os.listdir()

['08_multitarget-small-models-crop.ipynb',
 'LICENSE',
 '.git',
 'TODO List.odt',
 '05_multitarget-model-full-dataset-sparse.ipynb',
 '01_basic-data-loading-all-frames-few-videos.ipynb',
 '06_multitarget-model-full-dataset-sparse-custom-splitter.ipynb',
 '04_multitarget-model_distr.ipynb',
 '03_multitarget-model.ipynb',
 'm2-multitarget-all-videos-sparse-model-preds.csv',
 'data',
 '02_basic-data-loading-few-frames-all-videos.ipynb',
 'Untitled.ipynb',
 '__pycache__',
 '07_multitarget-small-models-squish.ipynb',
 '01_data-prep.ipynb',
 'abc.csv',
 'README.md',
 'images_df.csv',
 'm2-multitarget-all-videos-sparse-model-preds-stratified.csv',
 '.gitignore',
 'login.py',
 'm1-multitarget.pkl',
 '.ipynb_checkpoints',
 '03_baseline-model.ipynb',
 'models',
 'q.csv']

In [ ]:
zf_path = "surgtoolloc2022_dataset.zip"

with zipfile.ZipFile(zf_path, 'r') as zip_ref:
     zip_ref.extractall()

shutil.move('_release/training_data/video_clips', 'video_clips')
shutil.move('_release/readme.txt', '.')
shutil.move('_release/training_data/labels.csv', '.')

In [6]:
video_files = get_files('data/video_clips')
len(video_files)

24695

In [7]:
video = Video.from_file(video_files[100])
video.layout.width='512px'
video

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x004\x8f\xc8mdat\x00\x…

In [8]:
len([i for i in range(1800) if i%15==0])*len(video_files)

2963400

In [31]:
def extract_images(vf):
    """ 
    writing frames into its own directory inside the train_images folder
    """
    dst = Path().absolute()/'data/train_images2'/vf.stem
    dst.mkdir(exist_ok=True) 
    vid = cv2.VideoCapture(str(Path().absolute()/vf)) #the video being called
    
    """
    choose specific frames
    """
    fns = [i for i in range(1801) if i%15==0]

    for fn in fns:
        vid.set(1,fn)                               # fn is the frame we want
        ret, f = vid.read()                         # read the frame
        if ret:
            name = str(dst/f'{fn:05}.jpg')              # create fully qualified frame name
            cv2.imwrite(name,f)                         # store the frame

    vid.release()
        

In [32]:
(Path().absolute()/'data/train_images2').mkdir(exist_ok=True)

In [33]:
%time parallel(extract_images, video_files, n_workers=100)

CPU times: user 39 s, sys: 9.26 s, total: 48.3 s
Wall time: 19h 40min 38s


(#24695) [None,None,None,None,None,None,None,None,None,None...]

In [ ]:
fs = get_image_files('train_images2')

In [ ]:
# def f(i): return PILImage.create(i).size
# dims = parallel(f, files[:100000], n_workers=50)
# pd.Series(dims).value_counts()

# Creating a tiny version of the data

In [7]:
path = Path("./data/train_images_tiny")
path.mkdir(exist_ok=True)

In [8]:
src = Path("./data/train_images_small")
dst = Path("./data/train_images_tiny")

In [23]:
# Copy two images per video clip for fast experimentation
for d in src.ls().sorted():
    (dst/d.name).mkdir(exist_ok=True)
    for f in d.ls().sorted()[:2]:
        shutil.copy(f,dst/d.name)